First check that you have pytorch installed, instructions are here: https://pytorch.org/get-started/locally/ , prefferably do it with anaconda if you can, I think that will lead to less problems down the road if we use other libraries.

If cuda isnt available and you have an nvidia gpu try to get that too: https://docs.nvidia.com/cuda/cuda-installation-guide-microsoft-windows/index.html

In [1]:
import pickle
import torch

from torch import FloatTensor, LongTensor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt

if not torch.cuda.is_available():
    print("if you have an nvidia gpu get the cuda core package")

ModuleNotFoundError: No module named 'torch'